In [1]:
#import what we need
from requests_html import HTMLSession
session = HTMLSession()
import pandas as pd
import newspaper
import json
import geopandas as gpd


#use session to get the page
r = session.get('https://news.google.com/search?q=agriculture%20telangana&hl=en-IN&gl=IN&ceid=IN%3Aen')  #agriculture telangana

#render the html, sleep=1 to give it a second to finish before moving on. scrolldown= how many times to page down on the browser, to get more results. 5 was a good number here
#r.html.render(sleep=1, scrolldown=5)      ----------------------# THIS LINE WAS FETCHING ME ERROR IN JUPYTER NOTEBOOK

#find all the articles by using inspect element and create blank list
articles = r.html.find('article')
newslist = []

#loop through each article to find the title and link. try and except as repeated articles from other sources have different h tags.
for item in articles:
    try:
        newsitem = item.find('h3', first=True)
        title = newsitem.text
        link = newsitem.absolute_links
        newsarticle = {
            'title': title,
            'link': link 
        }
        newslist.append(newsarticle)
    except:
       pass

#print the length of the list
print('List of news articles: ',len(newslist))


List of news articles:  100


In [2]:
#Create a dataframe of newslist and link
df = pd.DataFrame(newslist)


#Scrape the list of article dictionary from news list
news_articles = []
for i in df.link:
    url = list(i)[0]
    article = newspaper.Article(url=url, language='en')
    article.download()
    article.parse()

    article ={
        "title": str(article.title),
        "text": str(article.text),
        "authors": article.authors,
        "published_date": str(article.publish_date),
        "top_image": str(article.top_image),
        "videos": article.movies,
        "keywords": article.keywords,
        "summary": str(article.summary)
    }
    news_articles.append(article)

In [3]:
#Convert list of news articles into dataframe with all scraped information
news = pd.DataFrame(news_articles)
text = news.text[0]

In [4]:
#Read geodataframe for mandal and district names to search in news articles
gdf = gpd.read_file('../DPPD/data/TSDM/Mandal_Boundary.shp')
mandal = list(gdf.Mandal_Nam.unique()) #unique mandals
dist = list(gdf.Dist_Name.unique())    #unique districts

In [6]:
# Access list of mandal mentione in each article

keyword_mandal = []
mandal_list = []

for text in  news.text:
    for i in range(0, len(mandal)):
        if mandal[i] in text:
            keyword_mandal.append(mandal[i])
    # insert the list to the set
        list_set = set(keyword_mandal)
    # convert the set to the list
        unique_list = (list(list_set))
    mandal_list.append(unique_list)


In [7]:
# Access list of district mentione in each article
keyword_dist = []
dist_list = []
for text in  news.text:
    for i in range(0, len(dist)):
        if dist[i] in text:
            keyword_dist.append(dist[i])
    # insert the list to the set
        list_set = set(keyword_dist)
    # convert the set to the list
        unique_list = (list(list_set))
    dist_list.append(unique_list)

In [8]:
#add mandal and districts to news dataframe
news['Mandals'] = mandal_list
news['Districts'] = dist_list

#convert full news dataframe to csv
news.to_csv('news_full.csv', index = False)

In [9]:
#Dataframe with few columns
news1 = news[['title','Mandals','Districts','published_date','text']]
news1.to_csv('news.csv', index = False)

In [10]:
news1

,title,Mandals,Districts,published_date,text
0,From paddy to palm oil: Telangana focuses on w...,[Suryapet],"[Hyderabad, Suryapet, Khammam]",2022-12-06 10:43:49+00:00,"After a push for paddy production, the Telanga..."
1,vice-chancellor appointment row: Telangana set...,[Suryapet],"[Hyderabad, Suryapet, Khammam]",None,HYDERABAD: With appointment of vice-chancellor...
2,‘Telangana’s agricultural output has increased...,[Suryapet],"[Hyderabad, Suryapet, Khammam]",2022-11-25 13:40:53+00:00,Hyderabad: Telangana Agriculture minister S Ni...
3,Telangana: Agricultural residential degree col...,"[Wanaparthy, Karimnagar, Suryapet]","[Suryapet, Khammam, Hyderabad, Wanaparthy, Kar...",2022-12-03 20:54:43+05:30,Telangana: Agricultural residential degree col...
4,Telangana govt to provide free EAMCET coaching...,"[Wanaparthy, Karimnagar, Suryapet]","[Suryapet, Khammam, Hyderabad, Wanaparthy, Kar...",2022-12-06 05:40:45+00:00,Hyderabad: Telangana government is getting rea...
...,...,...,...,...,...
95,Telangana International Seed Testing Lab gets ...,"[Hayathnagar, Ramagundam, Wankidi, Srirangapur...","[Hyderabad, Rangareddy, Sangareddy, Vikarabad,...",2022-08-27 23:58:51+05:30,Telangana International Seed Testing Lab gets ...
96,Telangana is better than Gujarat in supplying ...,"[Hayathnagar, Ramagundam, Wankidi, Srirangapur...","[Hyderabad, Rangareddy, Sangareddy, Vikarabad,...",None,By Express News Service\n\nHYDERABAD: Telangan...
97,"Gujarat polls: As joblessness bothers BJP, it ...","[Hayathnagar, Ramagundam, Wankidi, Srirangapur...","[Hyderabad, Rangareddy, Sangareddy, Vikarabad,...",2022-12-03 15:19:21+05:30,"Gujarat polls: As joblessness bothers BJP, it ..."
98,Asifabad: Tiger continues to roam in rural par...,"[Hayathnagar, Ramagundam, Wankidi, Srirangapur...","[Hyderabad, Rangareddy, Sangareddy, Vikarabad,...",2022-11-19 20:50:09+05:30,Asifabad: Tiger continues to roam in rural par...
